# Task is to Reading mail from Outlook
### Note : Only for Reply.
### 1. Connecting Notebook with Outlook
### 2. Reading mail from category 1 folder, fetching body of mail only and storing into CSV
### 3. Reading mail from category 2 folder, fetching body of mail only and storing into CSV

# Importing Libraries

In [1]:
# Library for reading mail and writing to csv
import unicodecsv as csv
import email
from bs4 import BeautifulSoup
from imap_tools import MailBox, AND, A
import re

import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns



# NLP LIBRARIES
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import *
from nltk.classify import NaiveBayesClassifier
from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score,classification_report
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB


# downloading stopwords corpus
nltk.download('stopwords')
#nltk.download('wordnet')
#stopwords = set(stopwords.words("english"))

# for showing all the plots inline
%matplotlib inline

#!pip install spacy

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mansurya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# FOR Reply Emails

In [2]:
# Saving mail to inbox_AR for Action required mails
output_file = open(r'E:\EmailTriage\EmailTriage_C_Operatn_5-5-21\validation_data\inbox_validation_reply.csv','wb')
#C:/Users/mansurya/Desktop/Email_triage/EmailTriage/EmailTriage/Labelling/Web_Labelling/inbox.csv
output_writer = csv.writer(output_file, encoding='utf-8') #, encoding='latin2',utf-8,delimiter=';',

In [3]:
# Folder names Emaila and Category
output_writer.writerow(["email","category"])      

16

In [4]:
email_list_AR = []
 
# get list of email subjects from INBOX folder
mailbox = MailBox('outlook.office365.com')
mailbox.login('afzal.mansury@centelon.com\operations@centelon.com', 'mpxbljwxbdzxbvzy', initial_folder='inbox')  # or mailbox.folder.set instead 3d arg

for msg in mailbox.fetch(A(seen=True)):
    soup = BeautifulSoup(msg.html, features="html.parser")
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out
    elements = soup.find_all("div", id="Signature")
    #elements = soup.find_all("div","Subject")

    for element in elements:
        element.decompose()

    if len(soup.get_text())>0:
        text = soup.get_text()
    else:
        soup = BeautifulSoup(msg.text)
        text = soup.get_text()
    
#     # kill all script and style elements
#     for script in soup(["script", "style"]):
#         script.extract()    # rip it out
#     elements = soup.find_all("div", id="Signature")
#     #elements = soup.find_all("div","Subject")

#     for element in elements:
#         element.decompose()
#     text = soup.get_text()
    #print("With beautiful soup", text)
    
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    subjects = msg.subject
    #print(subjects)
    #print("Origianl mail--------++++++",text)
    
    
    match = re.search(r'((Re\:\sFW\:)|(Re\:)|(RE\:)|(FW\:)|(Fw\:)|(Fwd\:)(RE\:\sFW\:))', subjects)
    #print("------------",text,"-------------")
    # If-statement after search() tests if it succeeded
    if match:
        print('found',match.group(0)) ## 'found word:cat'
        #print(text)
        if match.group()=="RE:" or match.group()=="Re:" or match.group()=="FW:" or match.group=="Fw:"or match.group()=="Fwd:":
            
            try:
                try:
                    pattern = re.compile(r'RE:|Re:|FW:|Fw:|Fwd:')
                    matches = pattern.finditer(text)
                    match_span_sub = []
                    for match in matches:
                        match_span_sub.append(match.span())
                    last_span_sub = match_span_sub[0] #[0]
                    print(len(match_span_sub))

                    text = text[last_span_sub:]
#                     text_full = []
#                     for text in text.splitlines():
#                         if not text.startswith(("RE:")): #"Subject:",
#                             text_full.append(text)
#                     text = " ".join(text_full)
                except:
                    text = text
                    #print(text)
            except:
                print("no subject found")
            try:
                match = re.search(r'<operations@centelon.com>|<future@centelon.com>|Thanking you,|Yours sincerely,|E:|Thanks and regards|Thanks and regards,|Best regards,|Best regards,|Thank you,|Thank You!|Thank you|Thanks,|Thanks & Regards,|From:|Thanks & regards,|Kindly regards,|Regards Poonam AR Team|Thanks & Regards|Signing off,|Regards|Best regards,|Kind regards,|Warm regards,|IMPORTANT: This e-mail|CONFIDENTIALITY AND PRIVACY NOTICE|CONFIDENTIALITY:|With kind regards,|Kind regards|THANKING YOU ALL|Yours sincerely &  Regards|Warm Regards,|THANKING YOU ALL|Kind regards', text)
                result = text.index(match.group(0))
                text = text[:result]
                text=re.sub("CAUTION: This email originated from outside of the organization. Do not click links or open attachments unless you recognize the sender and know the content is safe.","",text,re.IGNORECASE)
                print("Try & Except 2 -----",text)
                email_list_AR.append(text)
                output_writer.writerow([text,"NAR"])
            except:
                print("Try & Except 3 _________", text)
                #print(text)
                email_list_AR.append(text)
                output_writer.writerow([text,"NAR"])
    else:
        #print(text)
        text_full = []
        for text in text.splitlines():
            if not text.startswith(("Subject:","Cc:","Sent:","From:","To:")): #"Subject:",
                text_full.append(text)
                
        text = " ".join(text_full)
        try:
            match = re.search(r'Thanks & Regards,|From:|Thanks & regards,|Kindly regards,|Regards Poonam AR Team|Thanks & Regards|Signing off,|Regards|Best regards,|Kind regards,|Warm regards,|IMPORTANT: This e-mail|CONFIDENTIALITY AND PRIVACY NOTICE|CONFIDENTIALITY:|With kind regards,|Kind regards|THANKING YOU ALL|Yours sincerely &  Regards|Warm Regards,|THANKING YOU ALL', text)
            result = text.index(match.group(0))
            text = text[:result]
            #print(text[:result])
            #print(" CISL2021125 mujhe python sikhna hai",text)
            #email_list_AR.append(text)
            #output_writer.writerow([text,"AR"])
        except:
            print("Not Required",text)
            #email_list_AR.append(text)
            #output_writer.writerow([text,"AR"])
    
    print("*"*40,"End of mail body","*"*40)
    
            
output_file.close()


found Re:
2
Try & Except 2 ----- Hi Team,
I have completed both the assessments. However, it is not clear how to schedule the 1 to 1 zoom session. Is there a link that was shared?

**************************************** End of mail body ****************************************
**************************************** End of mail body ****************************************
found RE:
1
Try & Except 2 ----- Please confirm whether you have booked or not booked? Please check your email.
Team Operations

**************************************** End of mail body ****************************************
Not Required DSN Your message to sarita.igale@centelon.com couldn't be delivered. sarita.igale wasn't found at centelon.com. operations Office 365 sarita.igale Action Required Recipient Unknown To address How to Fix It The address may be misspelled or may not exist. Try one or more of the following: Send the message again following these steps: In Outlook, open this non-delivery report (NDR

found RE:
Try & Except 2 ----- Hi Pragati,
Thanks for raising the query.
Any information about tracking the candidates you referred, Sharing the resumes  or referral bonus you can always
get in touch with Chinmay as he looks after Recruitment.

**************************************** End of mail body ****************************************
found RE:
2
Try & Except 2 ----- Currently no project

**************************************** End of mail body ****************************************
found RE:
2
Try & Except 2 ----- Hi Mathew,
On the induction day we have taken them through centor and explained about the Leaves and attendance as well.
However for any doubts related to centor will surely help them and get it resolved.
Thanks
Chinmay P

**************************************** End of mail body ****************************************
found Re:
Try & Except 2 ----- Done!
Both test and interview schedule.

**************************************** End of mail body *****************

**************************************** End of mail body ****************************************
found RE:
1
Try & Except 2 ----- Any update on this?

**************************************** End of mail body ****************************************
found Re:
Try & Except 2 ----- Hi Srividya,
To whom she is going to handover her laptop and IT asset ?

**************************************** End of mail body ****************************************
found RE:
1
Try & Except 2 ----- Hello Team,
Good Noon .
As Megha is out of Pune and cannot handover laptop now; she will return laptop post situations are fine from her side .

**************************************** End of mail body ****************************************
found Fw:
**************************************** End of mail body ****************************************
found Re:
2
Try & Except 2 ----- Hi Team,
I have recieved handover sign off from everyone and from Suma on email. Attaching the email for the same here along w

found RE:
Try & Except 2 ----- Approved
Thanks and 
**************************************** End of mail body ****************************************
Not Required Hi, I’ve booked the flight ticket for Mumbai from https://www.manntravel.com.au/ and request for the reimbursement amount that I paid which is $1595/-. PFA. Thanks Amit Visapurkar
**************************************** End of mail body ****************************************
found Re:
Try & Except 2 ----- Hi team - is the offer released ?

**************************************** End of mail body ****************************************
found RE:
1
Try & Except 2 ----- Hi Sri,
Please release the offer by today. I have a discussion with candidate tomorrow morning for a potential project role.

**************************************** End of mail body ****************************************
**************************************** End of mail body ****************************************
found Re:
Try & Except 2 ----- Hi T

found RE:
Try & Except 2 ----- Approved.

**************************************** End of mail body ****************************************
**************************************** End of mail body ****************************************
found RE:
Try & Except 2 ----- Hi Varun,
We have create email account and AD credentials. Will share above details with you shortly.
Name: Snigdha Singh
Email  Id’s: Snigdha.Singh@centelon.com
User id: singhs
Thanks.
Santosh shinde

**************************************** End of mail body ****************************************
found Re:
1
Try & Except 2 ----- Hi Team,
Please update a username in centor record.
username: singhsn

**************************************** End of mail body ****************************************
**************************************** End of mail body ****************************************
**************************************** End of mail body ****************************************
found Re:
5
Try & Except 2 

...

Not Required Hi sir, This is k.madhu vikas ,want to inform you that my personal laptop got crashed 2days ago , I want to request you that could you please provide me office laptop so that i can complete my hands-on challenges and training without getting interrupted.please help me with this situation. Thank you. Get Outlook for Android
**************************************** End of mail body ****************************************
Not Required Dear Treesa, Congratulations!! On behalf of Centelon, we are delighted to enclose the offer letter. Please review and provide your formal acceptance by signing the pages of the documents and emailing a scanned copy of the letter at the earliest. Please feel free to get back to us if you have any queries. We are excited to have you on-board! E: Future@centelon.com
**************************************** End of mail body ****************************************
found Re:
Try & Except 2 ----- I did not update. Can i update it?

******************

found Re:
2
Try & Except 2 ----- Looks good 
**************************************** End of mail body ****************************************
found RE:
1
Try & Except 2 ----- Hi Sachin.
Good Afternoon,
Since
Sujit Ladole has not completed 6 months of his employment as on 31st March 2021 we have marked his "Eligibility For Promotion/Increment" as No.

**************************************** End of mail body ****************************************
found FW:
1
Try & Except 2 ----- FYI

**************************************** End of mail body ****************************************
Not Required Hi Team, Wanted to inform that today is the last working day for Divya V. We have the company Laptop with her, we need to recover from her.  Considering the Lockdown, please advise on the nexts steps. Let me know any other procedures that we need to follow in relieving her. John
**************************************** End of mail body ****************************************
*****************

Not Required Dear Ramya, Congratulations!! On behalf of Centelon, we are delighted to enclose the offer letter. Please review and provide your formal acceptance by signing the pages of the documents and emailing a scanned copy of the letter at the earliest. Please feel free to get back to us if you have any queries. We are excited to have you on-board! E: Future@centelon.com
**************************************** End of mail body ****************************************
Not Required Dear Ramya, Congratulations!! On behalf of Centelon, we are delighted to enclose the offer letter. Please review and provide your formal acceptance by signing the pages of the documents and emailing a scanned copy of the letter at the earliest. Please feel free to get back to us if you have any queries. We are excited to have you on-board! E: Future@centelon.com
**************************************** End of mail body ****************************************
found Re:
Try & Except 2 ----- 
Hi Srividya,
P

found Re:
Try & Except 2 ----- Hi Team,
I have created an O365 email account and credentials, I will share details with the user's personal e-mail Id.
Name:  Athira K
email Id: athira.k@centelon.com
Centor id:  athirak
--

**************************************** End of mail body ****************************************
found FW:
Try & Except 2 ----- Aysha, please release an offer letter asap with below mail details. Designation is consultant and other change is, in the format you have please
change all Centelon Solutions to Centelon PTY LTD. Basically take out the word solutions everywhere in the contract. This is because this employed has to be under Centelon Pty entity.
Also have to take out RSU clause and Bonus clause

**************************************** End of mail body ****************************************
**************************************** End of mail body ****************************************
Not Required Dear Vishal, Congratulations!! On behalf of Centelon, we

found Re:
Try & Except 2 ----- 
Hi Team,
Thanks a lot for the opportunity and I accept this offer.
Please find the attached document which has my Electronic Signature to confirm that I have accepted the offer.
Please let me know if anything else is needed from my side.

**************************************** End of mail body ****************************************
found RE:
Try & Except 2 ----- Dear Mary
Lakshman’s account has been created and login credentials sent to his personal ID
Email:  lakshman.tanneru@Centelon.com
Centor ID:
tannerul

**************************************** End of mail body ****************************************
found Re:
1
Try & Except 2 ----- Hi Team,
Athira's login successfully into office 365 accounts and teams. If you have any query's please feel free to contact us. Clear centor error massege.
--

**************************************** End of mail body ****************************************
found Re:
1
Try & Except 2 ----- Hi Team,
Ramya's login 

In [5]:
len(email_list_AR)

274

In [ ]:
# email_list_AR = []
 
# # get list of email subjects from INBOX folder
# mailbox = MailBox('outlook.office365.com')
# mailbox.login('afzal.mansury@centelon.com', 'mpxbljwxbdzxbvzy', initial_folder='test_mail/NAR')  # or mailbox.folder.set instead 3d arg

# for msg in mailbox.fetch(A(seen=True),mark_seen = False):
#     soup = BeautifulSoup(msg.html, features="html.parser")
    
#     # kill all script and style elements
#     for script in soup(["script", "style"]):
#         script.extract()    # rip it out
#     elements = soup.find_all("div", id="Signature")
#     #elements = soup.find_all("div","Subject")
    
#     for element in elements:
#         element.decompose()
#     text = soup.get_text()
    
#     # break into lines and remove leading and trailing space on each
#     lines = (line.strip() for line in text.splitlines())
#     # break multi-headlines into a line each
#     chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
#     # drop blank lines
#     text = '\n'.join(chunk for chunk in chunks if chunk)
#     subjects = msg.subject
#     print(subjects)
    
    
#     match = re.search(r'((Re\:\sFW\:)|(Re\:)|(FW\:)|(RE\:)|(RE\:\sFW\:))', subjects)
#     #print("------------",text,"-------------")
#     # If-statement after search() tests if it succeeded
#     if match:
#         #print('found',match.group(0)) ## 'found word:cat'
#         if match.group()=="FW:":
#             try:
#                 pattern = re.compile(r'Subject:')
#                 matches = pattern.finditer(text)
#                 match_span_sub = []
#                 for match in matches:
#                     match_span_sub.append(match.span())
#                 last_span_sub = match_span_sub[-1][0]
#                 print(last_span_sub)
                
#                 text = text[last_span_sub:]
#                 text_full = []
#                 for text in text.splitlines():
#                     if not text.startswith(("Subject:")): #"Subject:",
#                         text_full.append(text)

#                 text = " ".join(text_full)
#                 try:
#                     match = re.search(r'Thanks & Regards,|Signing off,|Regards|Kind regards,|CONFIDENTIALITY:|With kind regards,|Kind regards|THANKING YOU ALL|Yours sincerely &  Regards|THANKING YOU ALL', text)
#                     result = text.index(match.group(0))
#                     text = text[:result]
#                     print(text)
#                     #email_list.append(text)
#                     #output_writer.writerow([text,"AR"])
#                 except:
#                     print(text)
#                     #email_list_AR.append(text)
#                     #output_writer.writerow([text,"AR"]
#             except:
#                 print("______")
#         else:
#             print('found',match.group(0))
#     else:
#         print("Not found")

# FOR NAR Emails

In [ ]:
output_file = open(r'C:/Users/mansurya/Desktop/Email_triage/test/inbox_NAR.csv','wb')
#C:/Users/mansurya/Desktop/Email_triage/EmailTriage/EmailTriage/Labelling/Web_Labelling/inbox.csv
output_writer = csv.writer(output_file, encoding='utf-8') #, encoding='latin2',utf-8,delimiter=';',

In [ ]:
output_writer.writerow(["email","category"])      

In [ ]:
email_list_NAR = []
 
# get list of email subjects from INBOX folder
mailbox = MailBox('outlook.office365.com')
mailbox.login('afzal.mansury@centelon.com\payments@centelon.com', 'mpxbljwxbdzxbvzy', initial_folder='inbox/NAR')  # or mailbox.folder.set instead 3d arg

for msg in mailbox.fetch(A(seen=True),mark_seen = False):
    soup = BeautifulSoup(msg.html, features="html.parser")
    
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out
    elements = soup.find_all("div", id="Signature")
    #elements = soup.find_all("div","Subject")
    
    for element in elements:
        element.decompose()
    text = soup.get_text()
    
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    subjects = msg.subject
    print(subjects)
    
    
    match = re.search(r'((Re\:\sFW\:)|(Re\:)|(Fw\:)|(FW\:)|(Fwd\:)|(RE\:)|(RE\:\sFW\:))', subjects)
    #print("------------",text,"-------------")
    # If-statement after search() tests if it succeeded
    if match:
        print('found',match.group(0)) ## 'found word:cat'
    else:
        #print(text)
        text_full = []
        for text in text.splitlines():
            if not text.startswith(("Subject:","Cc:","Sent:","From:","To:")): #"Subject:",
                text_full.append(text)
                
        text = " ".join(text_full)
        try:
            match = re.search(r'Thanks & Regards,|Regards,|Kind regards,|CONFIDENTIALITY:|With kind regards,|Kind regards|THANKING YOU ALL|Yours sincerely &  Regards|THANKING YOU ALL', text)
            result = text.index(match.group(0))
            text = text[:result]
            print(text[:result])
            email_list_NAR.append(text)
            output_writer.writerow([text,"NAR"])
        except:
            print(text)
            email_list_NAR.append(text)
            output_writer.writerow([text,"NAR"])
    
    print("*"*40,"End of mail body","*"*40)
    
            
output_file.close()
#Kind regards


#     match = re.search(r'Subject:\s(RE\:|Re\:|FW\:|Fw\:|Re\:\sFW\:)', text)
# r'Subject:\s((Re\:\sFW\:)|(Re\:)|(FW\:)|(Fw\:)|(RE\:))|Re\:\sFW\:
# |Best regards,(.*?)|Best wishes,(.*?)|Fond regards,(.*?)|'
#                                         r'Kind regards(.*?)|Regards,(.*?)|Sincerely,(.*?)|Sincerely yours,(.*?)|'
#                                         r'Thank you,(.*?)|With appreciation,(.*?)|Yours sincerely,(.*?)

In [ ]:
len(email_list_NAR)

In [ ]:
print(len(email_list_AR),len(email_list_NAR))